In [1]:
import os
import glob
import pandas as pd
from collections import defaultdict

INPUT_DIR = "data"
OUTPUT_DIR = "data2"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# pattern: TICKER_options_daily_YYYY-MM-DD_YYYY-MM-DD.csv
pattern = os.path.join(INPUT_DIR, "*_options_daily_*.csv")
files = glob.glob(pattern)
print(f"Found {len(files)} files")

# group by ticker
by_ticker = defaultdict(list)
for f in files:
    base = os.path.basename(f)
    if "_options_daily_" not in base:
        continue
    ticker = base.split("_options_daily_")[0]
    by_ticker[ticker].append(f)

print(f"Detected {len(by_ticker)} tickers:", sorted(by_ticker.keys()))

for ticker, flist in sorted(by_ticker.items()):
    flist = sorted(flist)
    dfs = []

    for path in flist:
        try:
            df = pd.read_csv(path)
        except Exception as e:
            print(f"[{ticker}] ❌ failed to read {path}: {e}")
            continue

        if "date_ny" not in df.columns:
            print(f"[{ticker}] ⚠️ {path} missing 'date_ny', skipping")
            continue

        df["date_ny"] = pd.to_datetime(df["date_ny"], errors="coerce").dt.date
        dfs.append(df)

    if not dfs:
        print(f"[{ticker}] ⚠️ no valid chunks, skipping")
        continue

    merged = pd.concat(dfs, ignore_index=True)
    merged = merged[merged["date_ny"].notna()]

    merged = merged.sort_values("date_ny")
    merged = merged.drop_duplicates(subset=["date_ny"], keep="last").reset_index(drop=True)

    out_path = os.path.join(OUTPUT_DIR, f"{ticker}_options_daily_merged.csv")
    merged.to_csv(out_path, index=False)

    print(f"[{ticker}] ✅ merged {len(flist)} files → {len(merged)} rows → {out_path}")

Found 105 files
Detected 13 tickers: ['AAPL', 'AMZN', 'AVGO', 'GOOG', 'JPM', 'LLY', 'META', 'MSFT', 'NVDA', 'SPY', 'TSLA', 'V', 'WMT']
[AAPL] ✅ merged 11 files → 1363 rows → data2\AAPL_options_daily_merged.csv
[AMZN] ✅ merged 8 files → 987 rows → data2\AMZN_options_daily_merged.csv
[AVGO] ✅ merged 3 files → 371 rows → data2\AVGO_options_daily_merged.csv
[GOOG] ✅ merged 7 files → 864 rows → data2\GOOG_options_daily_merged.csv
[JPM] ✅ merged 11 files → 1363 rows → data2\JPM_options_daily_merged.csv
[LLY] ✅ merged 11 files → 1363 rows → data2\LLY_options_daily_merged.csv
[META] ✅ merged 7 files → 850 rows → data2\META_options_daily_merged.csv
[MSFT] ✅ merged 11 files → 1362 rows → data2\MSFT_options_daily_merged.csv
[NVDA] ✅ merged 3 files → 371 rows → data2\NVDA_options_daily_merged.csv
[SPY] ✅ merged 11 files → 1351 rows → data2\SPY_options_daily_merged.csv
[TSLA] ✅ merged 7 files → 862 rows → data2\TSLA_options_daily_merged.csv
[V] ✅ merged 11 files → 1363 rows → data2\V_options_daily_

In [3]:
import os
import pandas as pd

MERGED_DIR = "data2"

CUTOFFS = {
    "SPY":   "2020-05-31",
    "AAPL":  "2020-09-01",
    "MSFT":  "2020-05-31",
    "NVDA":  "2024-06-12",
    "AMZN":  "2022-06-07",
    "META":  "2022-06-11",
    "AVGO":  "2024-07-16",
    "TSLA":  "2022-08-26",
    "WMT":   "2024-02-27",
    "JPM":   "2020-05-31",
    "GOOG":  "2022-07-19",
    "V":     "2020-05-31",
    "LLY":   "2020-05-31"
}

DEFAULT_CUTOFF = "2020-05-31"
KEY_COLS = [
    "S_close",
    "total_vol",
    "avg_iv",
    "avg_vega",
    "call_vol",
    "put_vol",
    "call_iv",
    "put_iv",
]

files = [f for f in os.listdir(MERGED_DIR) if f.endswith("_options_daily_merged.csv")]
print(f"Found {len(files)} merged files to clean")

for fname in sorted(files):
    path = os.path.join(MERGED_DIR, fname)
    ticker = fname.split("_options_daily_merged.csv")[0]

    try:
        df = pd.read_csv(path)
    except Exception as e:
        print(f"[{ticker}] ❌ failed to read merged file: {e}")
        continue

    if "date_ny" not in df.columns:
        print(f"[{ticker}] ⚠️ no date_ny in merged file, skipping")
        continue

    df["date_ny"] = pd.to_datetime(df["date_ny"], errors="coerce").dt.date
    df = df[df["date_ny"].notna()]

    cutoff_str = CUTOFFS.get(ticker, DEFAULT_CUTOFF)
    cutoff = pd.to_datetime(cutoff_str).date()
    df = df[df["date_ny"] >= cutoff]

    existing_keys = [c for c in KEY_COLS if c in df.columns]
    if existing_keys:
        df = df.dropna(subset=existing_keys, how="all")

    df = df.sort_values("date_ny").drop_duplicates(subset=["date_ny"], keep="last").reset_index(drop=True)

    if df.empty:
        print(f"[{ticker}] ⚠️ all rows removed after cleaning; leaving file unchanged")
        continue

    df.to_csv(path, index=False)
    print(f"[{ticker}] ✅ cleaned → {len(df)} rows ≥ {cutoff}")

Found 13 merged files to clean
[AAPL] ✅ cleaned → 1298 rows ≥ 2020-09-01
[AMZN] ✅ cleaned → 854 rows ≥ 2022-06-07
[AVGO] ✅ cleaned → 326 rows ≥ 2024-07-16
[GOOG] ✅ cleaned → 826 rows ≥ 2022-07-19
[JPM] ✅ cleaned → 1363 rows ≥ 2020-05-31
[LLY] ✅ cleaned → 1363 rows ≥ 2020-05-31
[META] ✅ cleaned → 849 rows ≥ 2022-06-11
[MSFT] ✅ cleaned → 1362 rows ≥ 2020-05-31
[NVDA] ✅ cleaned → 348 rows ≥ 2024-06-12
[SPY] ✅ cleaned → 1351 rows ≥ 2020-05-31
[TSLA] ✅ cleaned → 796 rows ≥ 2022-08-26
[V] ✅ cleaned → 1363 rows ≥ 2020-05-31
[WMT] ✅ cleaned → 422 rows ≥ 2024-02-27


In [4]:
import os
import pandas as pd

# --- CONFIG ---
INPUT_FOLDER = "data2"
OUTPUT_FILE = os.path.join(INPUT_FOLDER, "master_dispersion_data.csv")

# --- Collect all merged ticker CSVs ---
csv_files = [
    f for f in os.listdir(INPUT_FOLDER)
    if f.endswith("_options_daily_merged.csv")
]

print(f"Found {len(csv_files)} ticker files.")

ticker_dfs = {}

for file in csv_files:
    ticker = file.split("_options_daily_merged")[0]
    path = os.path.join(INPUT_FOLDER, file)

    df = pd.read_csv(path)
    if "date_ny" not in df.columns:
        print(f"[{ticker}] ⚠️ missing date_ny column, skipping")
        continue

    df["date_ny"] = pd.to_datetime(df["date_ny"], errors="coerce")
    df = df[df["date_ny"].notna()]

    required = ["total_vol", "avg_vega", "avg_iv"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        print(f"[{ticker}] ⚠️ missing columns {missing}, skipping")
        continue

    # compute Vs here before merge
    df["Vs"] = df["total_vol"] * df["avg_vega"]

    # clean + sort
    df = df.sort_values("date_ny").drop_duplicates("date_ny", keep="last").reset_index(drop=True)
    ticker_dfs[ticker] = df

print(f"Loaded {len(ticker_dfs)} tickers: {sorted(ticker_dfs.keys())}")

# --- Build full date range ---
full_range = pd.date_range(start="2020-06-01", end="2025-10-30", freq="D")
master = pd.DataFrame({"date_ny": full_range})

# --- Order tickers: SPY first, then others by earliest start ---
start_dates = {t: ticker_dfs[t]["date_ny"].min() for t in ticker_dfs}
ordered_tickers = ["SPY"] + sorted(
    [t for t in ticker_dfs.keys() if t != "SPY"],
    key=lambda x: start_dates[x]
)

print("Merge order:", ordered_tickers)

# --- Merge each ticker into master ---
for ticker in ordered_tickers:
    df = ticker_dfs[ticker][["date_ny", "total_vol", "avg_vega", "Vs", "avg_iv"]].copy()
    df = df.rename(columns={
        "total_vol": f"{ticker}_vol",
        "avg_vega": f"{ticker}_avgVega",
        "Vs": f"{ticker}_Vs",
        "avg_iv": f"{ticker}_IV",
    })
    master = master.merge(df, on="date_ny", how="left")

# --- Drop rows where all ticker columns are NaN ---
ticker_cols = [col for col in master.columns if col != "date_ny"]
master = master.dropna(subset=ticker_cols, how="all")

# --- Save output ---
master.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Master dataset saved → {OUTPUT_FILE}")
print(f"Rows: {len(master)}, Columns: {len(master.columns)}")

Found 13 ticker files.
Loaded 13 tickers: ['AAPL', 'AMZN', 'AVGO', 'GOOG', 'JPM', 'LLY', 'META', 'MSFT', 'NVDA', 'SPY', 'TSLA', 'V', 'WMT']
Merge order: ['SPY', 'JPM', 'LLY', 'MSFT', 'V', 'AAPL', 'AMZN', 'META', 'GOOG', 'TSLA', 'WMT', 'NVDA', 'AVGO']
✅ Master dataset saved → data2\master_dispersion_data.csv
Rows: 1363, Columns: 53


In [5]:
import os
import pandas as pd

INPUT_FOLDER = "data2"
MASTER_FILE = os.path.join(INPUT_FOLDER, "master_dispersion_data.csv")

# --- load master ---
df = pd.read_csv(MASTER_FILE)

# parse dates as day-first with /, allowing 1- or 2-digit day/month
# format="mixed" lets pandas handle mixed "6/1/2020", "06/01/2020", etc.
df["date_ny"] = pd.to_datetime(
    df["date_ny"],
    format="mixed",
    dayfirst=True,
    errors="coerce"
)

if df["date_ny"].isna().sum() > 0:
    print("⚠️ Some date_ny entries could not be parsed. Check those rows manually.")

# --- identify columns ---
vs_cols = [c for c in df.columns if c.endswith("_Vs")]
iv_cols = [c for c in df.columns if c.endswith("_IV")]

# single-stock = everything except SPY
single_vs_cols = [c for c in vs_cols if not c.startswith("SPY_")]
single_iv_cols = [c for c in iv_cols if not c.startswith("SPY_")]

# --- aggregates (excluding SPY) ---
# 1) how many single stocks have data (Vs not NaN)
df["Singles_count"] = df[single_vs_cols].notna().sum(axis=1)

# 2) sum of Vs across single stocks
df["Singles_Vs_sum"] = df[single_vs_cols].sum(axis=1, skipna=True)

# 3) average Vs across single stocks with data
df["Singles_Vs_avg"] = df[single_vs_cols].mean(axis=1, skipna=True)

# 4) average IV across single stocks with data
df["Singles_IV_avg"] = df[single_iv_cols].mean(axis=1, skipna=True)

# --- reorder columns: date + 4 aggregates first ---
front_cols = ["date_ny", "Singles_count", "Singles_Vs_sum", "Singles_Vs_avg", "Singles_IV_avg"]
other_cols = [c for c in df.columns if c not in front_cols]
df = df[front_cols + other_cols]

# --- write dates back as DD/MM/YYYY with "/" ---
df["date_ny"] = df["date_ny"].dt.strftime("%d/%m/%Y")

df.to_csv(MASTER_FILE, index=False)

print("✅ Updated master_dispersion_data.csv with Singles_* fields and clean DD/MM/YYYY dates.")

✅ Updated master_dispersion_data.csv with Singles_* fields and clean DD/MM/YYYY dates.


In [6]:
import pandas as pd
import numpy as np
import os

INPUT_FOLDER = "data2"
OUTPUT_FOLDER = "data2"

MASTER_FILE = os.path.join(INPUT_FOLDER, "master_dispersion_data.csv")
OUT_FILE = os.path.join(OUTPUT_FOLDER, "dispersion_factors.csv")

# --- Load master ---
df = pd.read_csv(MASTER_FILE)
df["date_ny"] = pd.to_datetime(df["date_ny"], dayfirst=True, errors="coerce")

# --- Drop rows with missing SPY data (IV or Vs) ---
df = df.dropna(subset=["SPY_IV", "SPY_Vs"]).reset_index(drop=True)

# --- Identify single-stock IV columns (exclude SPY) ---
stock_iv_cols = [c for c in df.columns if c.endswith("_IV") and not c.startswith("SPY_")]

# --- F1: vega-exposure share (singles vs total) ---
df["F1"] = df["Singles_Vs_sum"] / (df["Singles_Vs_sum"] + df["SPY_Vs"])

# --- F2: IV spread (singles minus SPY) ---
df["F2"] = df["Singles_IV_avg"] - df["SPY_IV"]

# --- F3: 1 - implied correlation (constant-corr, equal-weight) ---
F3 = []
for _, row in df.iterrows():
    sigma_index = row["SPY_IV"]
    sigmas = row[stock_iv_cols].dropna().values
    N = len(sigmas)
    if N < 2:
        F3.append(np.nan)
        continue

    sum_sigma = np.sum(sigmas)
    sum_sigma_sq = np.sum(sigmas ** 2)
    numerator = (N ** 2) * (sigma_index ** 2) - sum_sigma_sq
    denominator = (sum_sigma ** 2) - sum_sigma_sq
    if denominator <= 0:
        F3.append(np.nan)
        continue
    rho_impl = numerator / denominator
    if rho_impl < -0.5 or rho_impl > 1.5:
        F3.append(np.nan)
        continue
    F3.append(1 - rho_impl)

df["F3"] = F3

# --- Save cleaned factors ---
df_out = df[["date_ny", "F1", "F2", "F3"]].copy()
df_out.to_csv(OUT_FILE, index=False)

print(f"✅ Dispersion factors saved to: {OUT_FILE}")
print(f"Total rows after dropping SPY-missing: {len(df_out)}")
print(df_out.head())
print(df_out.tail())

✅ Dispersion factors saved to: data2\dispersion_factors.csv
Total rows after dropping SPY-missing: 1348
     date_ny        F1        F2        F3
0 2020-06-01  0.162529  0.105292  0.712246
1 2020-06-02  0.222298  0.114830  0.785626
2 2020-06-03  0.108393  0.121592  0.795996
3 2020-06-04  0.056167  0.103976  0.726426
4 2020-06-05  0.180939  0.133809  0.986852
        date_ny        F1        F2        F3
1343 2025-10-24  0.369670  0.363109  1.062936
1344 2025-10-27  0.648049  0.479351  1.073749
1345 2025-10-28  0.761703  0.489011  1.035718
1346 2025-10-29  0.720832  0.601174  1.049872
1347 2025-10-30  0.473387  0.395753  1.026657


In [7]:
import os
import pandas as pd
import numpy as np

INPUT_FOLDER = "data2"
FACTORS_FILE = os.path.join(INPUT_FOLDER, "dispersion_factors.csv")

df = pd.read_csv(FACTORS_FILE)

# --- ensure numeric for core factors ---
for col in ["F1", "F2", "F3"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --- rolling 60d z-scores on F1, F2, F3 (use existing row order as time) ---
window = 60
min_pts = 40

def rolling_z(series, win=60, min_pts=40):
    m = series.rolling(win, min_periods=min_pts).mean()
    s = series.rolling(win, min_periods=min_pts).std()
    return (series - m) / s

df["F1_z60"] = rolling_z(df["F1"], window, min_pts)
df["F2_z60"] = rolling_z(df["F2"], window, min_pts)
df["F3_z60"] = rolling_z(df["F3"], window, min_pts)

# --- weighted composite of standardized factors ---
w1, w2, w3 = 0.2, 0.4, 0.4
df["Dispersion_Z_60d"] = (
    w1 * df["F1_z60"] +
    w2 * df["F2_z60"] +
    w3 * df["F3_z60"]
)

# --- EWMA variants on composite (20, 30, 60) ---
# Require at least `span` observations before emitting a value
for span in [20, 30, 60]:
    df[f"Dispersion_Z_EWMA_{span}"] = df["Dispersion_Z_60d"].ewm(
        span=span,
        adjust=False,
        min_periods=span
    ).mean()

# --- SMA variants on composite (60, 90, 120) ---
# Only produce values once full lookback is available
for win in [60, 90, 120]:
    df[f"Dispersion_Z_SMA_{win}"] = df["Dispersion_Z_60d"].rolling(
        window=win,
        min_periods=win
    ).mean()

# --- save back WITHOUT touching date_ny or order ---
df.to_csv(FACTORS_FILE, index=False)

print("✅ Saved Dispersion_Z_60d + EWMA(20,30,60) + SMA(60,90,120)")
print(df[[
    "date_ny",
    "Dispersion_Z_60d",
    "Dispersion_Z_EWMA_20",
    "Dispersion_Z_EWMA_30",
    "Dispersion_Z_EWMA_60",
    "Dispersion_Z_SMA_60",
    "Dispersion_Z_SMA_90",
    "Dispersion_Z_SMA_120"
]].tail())

✅ Saved Dispersion_Z_60d + EWMA(20,30,60) + SMA(60,90,120)
         date_ny  Dispersion_Z_60d  Dispersion_Z_EWMA_20  \
1343  2025-10-24          1.582014             -0.083583   
1344  2025-10-27          2.726209              0.184016   
1345  2025-10-28          2.427981              0.397727   
1346  2025-10-29          2.549316              0.602640   
1347  2025-10-30          0.768905              0.618475   

      Dispersion_Z_EWMA_30  Dispersion_Z_EWMA_60  Dispersion_Z_SMA_60  \
1343             -0.055029              0.037928             0.136448   
1344              0.124406              0.126068             0.160770   
1345              0.273023              0.201541             0.177765   
1346              0.419881              0.278517             0.220929   
1347              0.442399              0.294595             0.222259   

      Dispersion_Z_SMA_90  Dispersion_Z_SMA_120  
1343             0.165955              0.129075  
1344             0.199259              0.